In [14]:
# Initial imports
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn import svm
import hvplot.pandas
import re
from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import MinMaxScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')


# Pull in data and set some initial variables.

In [15]:
# Read in the daily data - Pulled from https://coincodex.com/crypto/bitcoin/historical-data/ 
data_path = Path("bitcoin_2010-8-16_2021-10-6.csv")
ohlcvm_df = pd.read_csv(
    data_path,
    index_col="Date",
    infer_datetime_format=True,
    parse_dates=True
)

ohlcvm_df.sort_index(ascending=True, inplace=True)

# Set initial capital for your portfolio
initial_capital = float(3000)

# Set the share size in bitcoins
share_size = .1

# Display sample data
ohlcvm_df.head()


,Open,High,Low,Close,Volume,Market Cap
Date,,,,,,
2010-08-16,0.07,0.07,0.07,0.07,0,0
2010-08-17,0.07,0.07,0.07,0.07,0,0
2010-08-18,0.07,0.07,0.07,0.07,0,0
2010-08-19,0.07,0.07,0.07,0.07,0,0
2010-08-20,0.07,0.07,0.07,0.07,0,0


## Calculate daily returns (pct_change())

In [16]:
# Calculate the daily returns using the closing prices and the pct_change function
ohlcvm_df["btc_dollar_returns"] = ohlcvm_df["Close"].pct_change()

# Drop all NaN values from the DataFrame
ohlcvm_df = ohlcvm_df.dropna()

# Review the DataFrame
display(ohlcvm_df.head())
display(ohlcvm_df.tail())

,Open,High,Low,Close,Volume,Market Cap,btc_dollar_returns
Date,,,,,,,
2010-08-17,0.07,0.07,0.07,0.07,0,0,0.0
2010-08-18,0.07,0.07,0.07,0.07,0,0,0.0
2010-08-19,0.07,0.07,0.07,0.07,0,0,0.0
2010-08-20,0.07,0.07,0.07,0.07,0,0,0.0
2010-08-21,0.07,0.07,0.07,0.07,0,0,0.0


,Open,High,Low,Close,Volume,Market Cap,btc_dollar_returns
Date,,,,,,,
2021-10-02,48147.57,48247.94,47467.84,47710.52,60614023324,900563548593,-0.007692
2021-10-03,47674.37,49163.40,47217.72,48144.15,47566767232,903971729927,0.009089
2021-10-04,48186.65,49425.78,47004.99,49176.76,60738746937,906668798743,0.021448
2021-10-05,49273.71,51785.32,49066.92,51545.05,68596958151,942701045366,0.048159
2021-10-06,51500.06,55556.80,50416.73,55385.45,77199761550,996348281001,0.074506


# Create Function to calculate the total portfolio value with short and long moving averages passed in.

In [17]:
# Create Function for cleaner code
def test_dataframe(temp_df, short_window, long_window):
    # Make a copy of the dataframe so it does not alter the original one
    ohlcvm_df_temp = temp_df.copy()
    
    # Create some strings for usage later as a dynamic column name 
    short_column = "sma_short_" + str(short_window) + '_' + str(long_window)
    long_column = "sma_long_" + str(long_window) + '_' + str(short_window)
    
    # Create the short term and long term moving averages and save them to the dynamic column names from above
    ohlcvm_df_temp[short_column] = ohlcvm_df_temp["Close"].rolling(window=short_window).mean()
    ohlcvm_df_temp[long_column] = ohlcvm_df_temp["Close"].rolling(window=long_window).mean()
    
    # Baseline Signal - 0.0 means do nothing, ie. hold
    ohlcvm_df_temp["signal"] = 0.0

    # create signals
    ohlcvm_df_temp["signal"][short_window:] = np.where(
        ohlcvm_df_temp[short_column][short_window:] > ohlcvm_df_temp[long_column][short_window:], 1.0, 0.0
    )
    # portfolio value = account_cash + btc_holding_value
    ohlcvm_df_temp["Entry/Exit"] = ohlcvm_df_temp["signal"].diff()

    # Buy a 500 share position when the dual moving average crossover Signal equals 1
    # Otherwise, `Position` should be zero (hold)
    ohlcvm_df_temp['BTC_portfolio'] = share_size * ohlcvm_df_temp['signal']

    # Determine the points in time where a 500 share position is bought or sold
    ohlcvm_df_temp['Entry/Exit Position'] = ohlcvm_df_temp['BTC_portfolio'].diff()

    # Multiply the close price by the number of shares held, or the Position
    ohlcvm_df_temp['Portfolio BTC Holdings'] = ohlcvm_df_temp['Close'] * ohlcvm_df_temp['BTC_portfolio']

    # Subtract the amount of either the cost or proceeds of the trade from the initial capital invested
    ohlcvm_df_temp['Portfolio Cash'] = initial_capital - (ohlcvm_df_temp['Close'] * ohlcvm_df_temp['Entry/Exit Position']).cumsum() 

    # Calculate the total portfolio value by adding the portfolio cash to the BTC holdings
    ohlcvm_df_temp['Portfolio Total'] = ohlcvm_df_temp['Portfolio Cash'] + ohlcvm_df_temp['Portfolio BTC Holdings']

    # Calculate the portfolio daily returns
    ohlcvm_df_temp['Portfolio Daily Returns'] = ohlcvm_df_temp['Portfolio Total'].pct_change()

    # Calculate the portfolio cumulative returns
    ohlcvm_df_temp['Portfolio Cumulative Returns'] = (1 + ohlcvm_df_temp['Portfolio Daily Returns']).cumprod() - 1
    
    # Choose the final Portfolio Value and create a small dict and then append that dict to the outer list we created above
    value = round(ohlcvm_df_temp['Portfolio Total'].iloc[-1], 2)
    key_name = str("Short_" + str(short_window) + '_' + "Long_" + str(long_window))    
    profit_dict = {"MA_combo": key_name, "MA_value":value}
    total_portfolio_value_list.append(profit_dict)
    
    # we actually don't need to re
    return ohlcvm_df_temp
    

## Prep the for loop and the outer list to be used to collate the results for each moving average combo

In [23]:
# Create the fast moving average column
# Define a window list of all the possible windows we want to try
window_short = [5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100]
window_long = [10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200]


# Pull out the portfolio total at the end and add to a list of totals for each iteration
total_portfolio_value_list = []

# nested for loop to allow us to test all valid possibilities for moving average combos
for short in window_short:
    ohlcvm_df_temp = 0
    for long in window_long:        
        if(short < long):
            ohlcvm_df_temp = 0
            temp_df = test_dataframe(ohlcvm_df, short, long)
        else:
            continue

total_portfolio_value_list_new = sorted(total_portfolio_value_list,key=lambda d: d['MA_value'], reverse=True)
print(temp_df)

                Open      High       Low     Close       Volume    Market Cap  \
Date                                                                            
2010-08-17      0.07      0.07      0.07      0.07            0             0   
2010-08-18      0.07      0.07      0.07      0.07            0             0   
2010-08-19      0.07      0.07      0.07      0.07            0             0   
2010-08-20      0.07      0.07      0.07      0.07            0             0   
2010-08-21      0.07      0.07      0.07      0.07            0             0   
...              ...       ...       ...       ...          ...           ...   
2021-10-02  48147.57  48247.94  47467.84  47710.52  60614023324  900563548593   
2021-10-03  47674.37  49163.40  47217.72  48144.15  47566767232  903971729927   
2021-10-04  48186.65  49425.78  47004.99  49176.76  60738746937  906668798743   
2021-10-05  49273.71  51785.32  49066.92  51545.05  68596958151  942701045366   
2021-10-06  51500.06  55556.

In [24]:
# take just the top performing combo by choosing the first index in the list (already sorted)
best_ma_combo = total_portfolio_value_list_new[0]
print(f"best_ma_combo: {best_ma_combo}")
# work to pull out the numbers from the MA_combo string, save as sh9ort and long variables to be used later
string = best_ma_combo['MA_combo']
short = string[6:8]
short = int(short)
long = string[-3:]
long = re.sub('[^0-9]','', long)
long = int(long)
print(f"short: {short}")
print(f"long: {long}")

best_ma_combo: {'MA_combo': 'Short_20_Long_80', 'MA_value': 9909.26}
short: 20
long: 80


In [25]:
## Call our function with the dataframe, optimal short and long moving average values passed in.

In [26]:
ohlcvm_df_temp = 0
# ohlcvm_df
df = test_dataframe(ohlcvm_df,int(short),int(long))
df

,Open,High,Low,Close,Volume,Market Cap,btc_dollar_returns,sma_short_20_80,sma_long_80_20,signal,Entry/Exit,BTC_portfolio,Entry/Exit Position,Portfolio BTC Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
Date,,,,,,,,,,,,,,,,,,
2010-08-17,0.07,0.07,0.07,0.07,0,0,0.000000,NaN,NaN,0.0,NaN,0.0,NaN,0.000,NaN,NaN,NaN,NaN
2010-08-18,0.07,0.07,0.07,0.07,0,0,0.000000,NaN,NaN,0.0,0.0,0.0,0.0,0.000,3000.000,3000.000,NaN,NaN
2010-08-19,0.07,0.07,0.07,0.07,0,0,0.000000,NaN,NaN,0.0,0.0,0.0,0.0,0.000,3000.000,3000.000,0.000000,0.000000
2010-08-20,0.07,0.07,0.07,0.07,0,0,0.000000,NaN,NaN,0.0,0.0,0.0,0.0,0.000,3000.000,3000.000,0.000000,0.000000
2010-08-21,0.07,0.07,0.07,0.07,0,0,0.000000,NaN,NaN,0.0,0.0,0.0,0.0,0.000,3000.000,3000.000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-02,48147.57,48247.94,47467.84,47710.52,60614023324,900563548593,-0.007692,44772.3605,43557.486625,1.0,0.0,0.1,0.0,4771.052,4370.717,9141.769,-0.004029,2.047256
2021-10-03,47674.37,49163.40,47217.72,48144.15,47566767232,903971729927,0.009089,44930.0670,43763.770375,1.0,0.0,0.1,0.0,4814.415,4370.717,9185.132,0.004743,2.061711
2021-10-04,48186.65,49425.78,47004.99,49176.76,60738746937,906668798743,0.021448,45038.2595,43986.160500,1.0,0.0,0.1,0.0,4917.676,4370.717,9288.393,0.011242,2.096131


### Plot the portfolio values over time based on the best moving average combo

In [27]:
# Visualize exit position relative to total portfolio value
exit = df[df['Entry/Exit'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='yellow',
    marker='v',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1200,
    height=500
)

# Visualize entry position relative to total portfolio value
entry = df[df['Entry/Exit'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='purple',
    marker='^',
    ylabel='Total Portfolio Value',
    width=1500,
    height=500
)

# Visualize the value of the total portfolio
total_portfolio_value = df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    xlabel='Date',
    width=1500,
    height=500
)

# Overlay the plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(
    title="BTC Algorithm - Total Portfolio Value",
    yformatter='%.0f'
)

:Overlay
   .Curve.I                    :Curve   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [Date]   (Portfolio Total)